In [40]:
from datetime import datetime
import tkinter as tk
from tkinter import messagebox

In [41]:
# Clase vértice - Usuários
class User:

  def __init__(self, user:str, email:str):

    """
    user: Usuário da rede
    email: email do usuário
    """
    self.user = user
    self.email = email
    self.hobbies = [] # Tags com interesses.

    
  def __str__(self): # Vai printar o objeto usuário como string
    return str(self.user)

  # Adiciona hobbies dos usuários  
  def addHobbie(self, hobbie):
    self.hobbies.append(hobbie)

In [42]:
# Classe Aresta - Conexões
class Conection:

  def __init__(self, user_1: str, user_2: str, n_interacoes: int):

    """
    User_1: Usuário seguidor
    User_2: Usuário que é seguido por User_1
    n_interacoes: Quantas vezes o usuário A curtiu, comentou ou compartilhou postagens do usuário B.
    """

    self.user_1 = user_1
    self.user_2 = user_2
    self.n_interacoes = n_interacoes

In [43]:
# Classe post
class Post:
    def __init__(self, id:int, author:User, content: str):
        self.id = id
        self.author = author
        self.content = content
        self.date = datetime.now()
        self.likes = set() # Usei o set para evitar duplicatas.

    def like(self, user: User):
        self.likes.add(user)

    def __str__(self):
        return f"Post {self.id} por {self.author.user}: {self.content}."

In [44]:
# Grafo - Unipoli
class UniPoli:

  def __init__(self):
    self.users = {} # Dicionário para armazenar usuários
    self.conections = [] # Lista para armazenar conexões
    self.posts = []  # Lista para armazenar post


  # Adiciona usuários
  def addUser(self, user, email):

    if user not in self.users:
      self.users[user] = User(user, email)


  # Adiciona conexões - Seguir
  def addConection(self, user_1, user_2, n_interacoes):

    if user_1 and user_2 in self.users:
      a = self.users[user_1]
      b = self.users[user_2]
      conexao = Conection(a, b, n_interacoes)
      self.conections.append(conexao)


  # Exclui usuário
  def delUser(self, user):
    if user in self.users:
      del self.users[user]
      # Apagar as arestas ligadas a user -  filtro
      self.conections = [a for a in self.conections if a.user_1.user != user and a.user_2.user != user]
      print(f'Usuário {user} removido com sucesso.')
      return True
    else:
        print(f'Usuário {user} não encontrado.')
        return False


  # Exclui aresta - Deixar de seguir
  def delConection(self, a, b):
    if a in self.users and b in self.users:
      for aresta in self.conections:
        if aresta.user_1.user == a and aresta.user_2.user == b:
          self.conections.remove(aresta)
          print(f'{a} deixou de seguir {b}.')
          return True
      print(f'{a} não segue {b}.')
    else:
      print(f'{a} or {b} não encontrado.')
    return False


    # Método postar
  def create_post(self, author_name, content: str):
    if author_name in self.users:
        author = self.users[author_name]
        post = Post(len(self.posts), author , content)
        self.posts.append(post)
        return post
    return None # Se autor não existe


  def feed(self, user_name: str):
    feed_posts = []
    if user_name not in self.users:
        return []

    current_user = self.users[user_name]

    # Econtrar todos os usuários que currente_user segue
    user_following = set()
    for conection in self.conections:
        if conection.user_1 == current_user:
            user_following.add(conection.user_2)

    # Gerar o feed
    for post in self.posts:
        if post.author == current_user:
            feed_posts.append(post)

        elif post.author in user_following:
            feed_posts.append(post)
            
    # Ordena post pelos mais recentes
    return sorted(feed_posts, key=lambda p: p.date, reverse=True)
    
      
  # Mostrar grafo
  def display(self):
    print('\n=============================== UniPoli ================================\n')
    print('Usuários: ')
    for user in self.users:
      print(f'{user} ', end=' ')

    print('\n \nConexões: ')
    for user in self.conections:
      print(f'{user.user_1} segue {user.user_2}. {user.user_1} interagiu {user.n_interacoes} vezes com o perfil de {user.user_2}.\n')

    # Printar os posts
    print('\nPosts: ')
    for post in self.posts:
        print(f'Post #{post.id} de {post.author.user}: {post.content}. {post.date}\n')


  # Recomendação para um usuário específico
  def recommend(self, user_ref):
    pass


  # Atualiza o número de interações
  def updateLike(self, user_1, user_2):
    # Procura a aresta existente
    for aresta in self.conections:
        if aresta.user_1.user == user_1 and aresta.user_2.user == user_2:
            aresta.n_interacoes += 1
            return True  # Atualizou com sucesso

    return False  # Usuário(s) inexistente(s)

In [45]:
## Teste
rede = UniPoli()

# Usuários
rede.addUser('Mikael', 'mikael@gmail.com')
rede.addUser('Madalena', 'Mad@gmail.com')
rede.addUser('Ana', 'Ana@gmail.com')
rede.addUser('Elon Musk', 'El@gmail.com')

# Conexões
rede.addConection('Mikael', 'Madalena', 1)
rede.addConection('Madalena', 'Mikael', 1)
rede.addConection('Madalena', 'Ana', 1)
rede.addConection('Ana', 'Madalena', 1)
rede.addConection('Mikael', 'Elon Musk', 100)

rede.display()

rede.updateLike('Mikael', 'Madalena')
rede.delConection('Mikael', 'Elon Musk')
rede.delConection('Mikael', 'Ana')
rede.delUser('Elon Musk')
rede.create_post('Mikael', 'Eu sou muito esperto!')
rede.create_post('Ana', 'Mikael é um gostoso!')
rede.create_post('Madalena', 'Mikael é super inteligente!')

rede.display()

# Testar o feed
# Feed de Mikael
print("Feed de Mikael:")
for post in rede.feed('Mikael'):
    print(f"- {post}")

print('\n')
print("Feed de Madalena:")
for post in rede.feed('Madalena'):
    print(f"- {post}")


=============================== UniPoli ================================

Usuários: 
Mikael  Madalena  Ana  Elon Musk  
 
Conexões: 
Mikael segue Madalena. Mikael interagiu 1 vezes com o perfil de Madalena.

Madalena segue Mikael. Madalena interagiu 1 vezes com o perfil de Mikael.

Madalena segue Ana. Madalena interagiu 1 vezes com o perfil de Ana.

Ana segue Madalena. Ana interagiu 1 vezes com o perfil de Madalena.

Mikael segue Elon Musk. Mikael interagiu 100 vezes com o perfil de Elon Musk.


Posts: 
Mikael deixou de seguir Elon Musk.
Mikael não segue Ana.
Usuário Elon Musk removido com sucesso.

=============================== UniPoli ================================

Usuários: 
Mikael  Madalena  Ana  
 
Conexões: 
Mikael segue Madalena. Mikael interagiu 2 vezes com o perfil de Madalena.

Madalena segue Mikael. Madalena interagiu 1 vezes com o perfil de Mikael.

Madalena segue Ana. Madalena interagiu 1 vezes com o perfil de Ana.

Ana segue Madalena. Ana interagiu 1 vezes com o per

In [46]:
class Gui:
    def __init__(self, master):
        self.master = master
        master.title("Modelo de Grafo do Instagram")

        master.geometry('600x600')
        
        self.rede = UniPoli() # Instancia seu modelo de grafo

        # Widgets da interface
        self.label = tk.Label(master, text="Bem-vindo ao UniPoli!")
        self.label.pack()

        # Botões, entradas de texto, etc.
        self.btn_add_user = tk.Button(master, text="Adicionar Usuário", command=self.open_add_user_window)
        self.btn_add_user.pack()

        self.btn_display_graph = tk.Button(master, text="Mostrar Grafo (Console)", command=self.display_graph_in_console)
        self.btn_display_graph.pack()

        # Adicione mais widgets para outras funcionalidades (Adicionar Conexão, Postar, Feed, etc.)

        self.close_button = tk.Button(master, text="Sair", command=master.destroy)
        self.close_button.pack()

    def open_add_user_window(self):
        # Janela para adicionar usuário
        add_user_window = tk.Toplevel(self.master)
        add_user_window.title("Adicionar Novo Usuário")

        add_user_window.geometry('600x600')

        tk.Label(add_user_window, text="Nome de Usuário:").pack()
        self.username_entry = tk.Entry(add_user_window)
        self.username_entry.pack()

        tk.Label(add_user_window, text="Email:").pack()
        self.email_entry = tk.Entry(add_user_window)
        self.email_entry.pack()

        tk.Button(add_user_window, text="Adicionar", command=self.add_user_action).pack()

    def add_user_action(self):
        try:
            username = self.username_entry.get()
            email = self.email_entry.get()

            if not username or not email:
                messagebox.showerror("Erro", "Nome de usuário e email não podem ser vazios.")
                return

            if username in self.rede.users:
                messagebox.showwarning("Aviso", f"Usuário '{username}' já existe.")
            else:
                self.rede.addUser(username, email)
                messagebox.showinfo("Sucesso", f"Usuário '{username}' adicionado com sucesso!")
                # Limpar campos após adicionar
                self.username_entry.delete(0, tk.END)
                self.email_entry.delete(0, tk.END)

        except Exception as e:
            messagebox.showerror("Erro", f"Ocorreu um erro: {e}")

    def display_graph_in_console(self):
        self.rede.display()
        messagebox.showinfo("Informação", "Detalhes do grafo foram impressos no console do Python.")

In [47]:
if __name__ == "__main__":
    root = tk.Tk()
    app = Gui(root)
    root.mainloop()


=============================== UniPoli ================================

Usuários: 

 
Conexões: 

Posts: 
